#### (Home credit competition 중심으로)
## 4.1 모델개발 - Sklearn model 
---
### 소스 및 데이터는 아래 kaggle을 참조하였습니다. 
* 캐글 주소 : https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction
* 해당 커널의 한글 번역은 파파고와 구글을 활용하였습니다.
___

In [1]:
# 라이브러리 
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

In [2]:
# 데이터 셋
app_train = pd.read_csv('d:/Projects/Cp/input/application_train.csv')
app_test = pd.read_csv('d:/Projects/Cp/input/application_test.csv')

In [3]:
#  레이블 인코딩 
le = LabelEncoder()
le_count = 0

for col in app_train:    
    if app_train[col].dtype == 'object':
        if len(list(app_train[col].unique())) <= 2: 
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col]  = le.transform(app_test [col])            
            le_count += 1

In [4]:
# 원 핫 인코딩
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [5]:
# 원 핫 인코딩 후 훈련/테스트 용 데이터 열(컬럼) 일치화

# TARGET(대출금 상환여부)열 따로 보관
train_labels = app_train['TARGET']

# 열(컬럼)기준 데이터 정렬하여 서로 없는 열(컬럼) 삭제
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# 학습용 데이터에 TARGET(대출금 상환여부)열 추가
app_train['TARGET'] = train_labels

In [6]:
# 취업일(DAYS_EMPLOYED) 이상치 데이터 처리

# 훈련용 셋
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# 테스트용 셋
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

In [7]:
#편의를 위해 DAYS_BIRTH의 절대값을 취하여 양수를 만듭니다
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

In [8]:
# 연령대 정보를 다른 데이터프레임에 저장하고 년으로 환산
age_data = app_train[['TARGET', 'DAYS_BIRTH']]
age_data['YEARS_BIRTH'] = age_data['DAYS_BIRTH'] / 365

# 나이 구간 설정
age_data['YEARS_BINNED'] = pd.cut(age_data['YEARS_BIRTH'], bins = np.linspace(20, 70, num = 11))

#### 기준선
* 단순한 기준에서 우리는 테스트 데이터의 모든 예제에 대해 동일한 값을 추측할 수 있다.
* 대출금을 제떼에 상환하지 않을 가능성을 예측해 달라는 요청을 받았으므로 만약 우리가 완전히 확신할 수 없다면 테스트 데이터의 모든 관측치에 대해 0.5를 추측할 것이다.
* 이러게 하면 경쟁에서 커브(ROC)가 0.5가 됩니다.
> (분류작업에서 무작위 추측이 0.5점을 받는다)
* 우리는 이미 어떤 점수를 받을 수 알고 있기 때문에 단순한 기준선 예측을 할 필요가 없다 
* 이제 실제 기준에 약간 더 정교한 로지스틱 회귀분석 모델을 사용한다.

#### Logistic Regression 
* 기준선을 얻으려면 범주형 변수를 인코딩한 후에 모든 피쳐들을 사용한다.
* 결측값을 채우고 범위를 정규화하여 데이터를 사전 처리한다.
* 다음 코드는 이러한 사전 처리 단계를 모두 수행한다.

In [9]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# 훈련용 데이터에서 대출금 상환여부(Target)열 제외 한 값을 train 데이터 프레임에 복사
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
    
features = list(train.columns)

# test 테이터프레임에 테스트용 데이터 복사
test = app_test.copy()

# 누락된 값을 중앙값으로 대체
imputer = Imputer(strategy = 'median')

# 각 피쳐를 0~1로 스케일
scaler = MinMaxScaler(feature_range = (0, 1))

# 훈련용 데이터에 모델 학습
imputer.fit(train)

# 훈련용 및 테스트용 데이터 모두 변환
train = imputer.transform(train)
test = imputer.transform(app_test)

# 스칼러 사용하여 반복
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 240)
Testing data shape:  (48744, 240)


* LogisticRegression 사용
* 기본 모델 설정에서 변경한 유일한 사항은 오버핏 양을 제어하는 정규화 매개 변수 C를 낮추는 것입니다
> 낮은 값은 오버핏을 감소
* 이렇게 하면 기본 LogisticRegression보다 약간 더 나은 결과를 얻을 수 있지만, 향후 모델에서는 여전히 낮은 수준을 유지할 수 있습니다.

In [10]:
from sklearn.linear_model import LogisticRegression

# 지정된 정규화 매개변수를 사용하여 모형 만들기
log_reg = LogisticRegression(C = 0.0001)

# 훈련용 데이터 학습
log_reg.fit(train, train_labels)

LogisticRegression(C=0.0001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

* 우리는 대출금을 갚지 않을 확률을 예측하기를 원하기 때문에 predict.proba 메쏘드를 사용합니다.
* m x 2 배열을 반환하는데 m은 관측치의 수입니다. 
* 첫 번째 열은 대상이 0(대출금 상환)일 확률이고 
* 두 번째 열은 대상이 1(대출금 미상환)일 확률입니다.
* (따라서 한 행의 경우 두 열의 합은 1이어야 함). 
* 대출금을 상환하지 않을 가능성을 원하므로 두 번째 열을 선택합니다.

In [11]:
# 예측 수행 후 두번째 열 선택
log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [12]:
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred

submit.head()

,SK_ID_CURR,TARGET
0,100001,0.087750
1,100005,0.163957
2,100013,0.110238
3,100028,0.076575
4,100038,0.154924


* 예측은 대출금이 상환되지 않을 확률이 0과 1 사이라는 것을 나타낸니다. 
* 이러한 예측을 사용하여 지원자를 분류하는 경우 대출이 위험하다는 것을 판단하는 확률 임계값을 설정할 수 있습니다.

In [13]:
# 제출물을 CSV 파일에 저장
submit.to_csv('log_reg_baseline.csv', index = False)